In [ ]:
%cd ..

# ImbizoPM Agents Test

This notebook demonstrates how to use the ImbizoPM agents to create and execute a project planning workflow.

In [ ]:
from imbizopm_agents.graph import create_project_planning_graph, run_project_planning_graph
from IPython.display import display, Image
from langgraph.graph.graph import CompiledGraph

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("ollama:llama3.3")

In [ ]:
# Create the project planning graph with checkpointing enabled
graph: CompiledGraph = create_project_planning_graph(llm, use_checkpointing=True)

In [ ]:
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Error creating graph: {e}")

## Running the Project Planning Process

We run the project planning graph with a sample input. The system will process the input through all the agents in the workflow and may request human assistance when needed.

In [ ]:
# Run the project planning process
results = run_project_planning_graph(
    graph,
    user_input="AgentFlow is a Python library that automates the orchestration of multi-step agent workflows by integrating intelligent planning, routing, and execution of specialized operations.",
    thread_id="demo-run-1"  # Adding a unique thread ID for this run
)

# Display the number of steps processed
print(f"Process completed with {len(results)} steps")

In [ ]:
# Example of how to extract final results
final_result = results[-1] if results else None

if final_result:
    print("\nFinal Project Plan Summary:")
    print(f"Idea: {final_result.get('idea', {}).get('refined', 'Not defined')}")
    print(f"Goals: {', '.join(final_result.get('goals', ['Not defined']))}")
    print(f"Deliverables: {len(final_result.get('deliverables', []))} items")
    print(f"Tasks: {len(final_result.get('tasks', []))} defined")
    
    # If the PMAdapter was reached, display export info
    if 'final_output' in final_result:
        print("\nExport Information:")
        print(f"Format: {final_result['final_output'].get('format', 'Unknown')}")
        print(f"Next Steps: {len(final_result['final_output'].get('next_steps', []))} items")

In [ ]:
# Interactive test with human input
new_project_idea = input("Enter a new project idea to plan: ")

if new_project_idea:
    print("\nProcessing your idea...\n")
    results = run_project_planning_graph(
        graph,
        user_input=new_project_idea,
        thread_id=f"user-input-{hash(new_project_idea)[:8]}"  # Generate a unique thread ID
    )
    
    # Display last message from the process
    if results and 'messages' in results[-1] and results[-1]['messages']:
        last_message = results[-1]['messages'][-1]
        print(f"\nFinal Response:\n{last_message.content}")